In [1]:
import os
import time

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from bclassification.utils_base import (
    print_class_weights,
    TrainingHistory,
    plot_metrics,
)
from bclassification.utils_fc import obs_to_vect_with_tc
from bclassification.utils_rnns import print_dataset
from experience import load_experience
from lib.action_space import is_do_nothing_action
from lib.constants import Constants as Const
from lib.data_utils import extract_history_windows
from lib.data_utils import make_dir, env_pf
from lib.dc_opf import TopologyConverter
from lib.tf_utils import (
    print_variables,
    MatthewsCorrelationCoefficient,
)
from lib.visualizer import Visualizer, pprint

tf.keras.backend.set_floatx("float64")
Visualizer()

experience_dir = make_dir(os.path.join(Const.RESULTS_DIR, "performance-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "bc-fc"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))
case, collector = load_experience(case_name, agent_name, experience_dir, env_dc=env_dc)


L2RPN_2019_ART (dc)


--------------------------------------------------------------------------------
                                        Loading Experience
--------------------------------------------------------------------------------
    - Loading chronics:                 ./results/performance-aug/l2rpn_2019_art-dc/agent-mip-chronic-****
    - Number of loaded chronics:        99


In [2]:
"""
    Parameters
"""

random_seed = 0

model_type = "res"  # "fc" or "res"

n_window_targets = 10
threshold = 0.5

dropout_rate = 0.2
n_hidden = 512
n_state = n_hidden

n_batch = 2
n_epochs = 500

downsampling_rate = 0.10

test_frac = 0.1
val_frac = 0.1 / (1 - test_frac)

max_chronic_len = 8064

In [6]:
"""
    Dataset
"""

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

obs_to_vect = obs_to_vect_with_tc(TopologyConverter(case.env))

X_all = []
Y_all = []
mask_all = []
mask_targets = []

for chronic_idx, chronic_data in collector.data.items():
    #     chronic_obses = chronic_data["obses"][:-1]
    #     chronic_labels = is_do_nothing_action(chronic_data["actions"], case.env, dtype=np.bool)

    chronic_obses = chronic_data["obses"][][:100]
    chronic_len = len(chronic_obses)
    
    chronic_labels = is_do_nothing_action(
        chronic_data["actions"][:chronic_len], case.env, dtype=np.bool
    )
    print(len(chronic_obses))
    print(len(chronic_labels))

    chronic_mask = np.array(
        [True] * chronic_len + [False] * (max_chronic_len - chronic_len)
    )

    chronic_obses_vect = np.vstack([obs_to_vect(obs) for obs in chronic_obses])
    chronic_padding_vect = np.zeros(
        ((max_chronic_len - chronic_len), chronic_obses_vect.shape[-1])
    )
    chronic_X_all = np.concatenate((chronic_obses_vect, chronic_padding_vect), axis=0)

    chronic_Y_all = np.zeros_like(chronic_mask, dtype=np.bool)
    chronic_Y_all[:chronic_len] = chronic_labels

    mask_positives = extract_history_windows(chronic_labels, n_window=n_window_targets)
    mask_negatives = np.logical_and(
        np.random.binomial(1, downsampling_rate, len(chronic_labels)).astype(np.bool),
        ~mask_positives,
    )
    chronic_mask_targets = np.zeros_like(chronic_mask, dtype=np.bool)
    chronic_mask_targets[:chronic_len] = np.logical_or(chronic_labels, mask_negatives)

    pprint(
        chronic_idx,
        chronic_obses_vect.shape,
        chronic_padding_vect.shape,
        chronic_X_all.shape,
        chronic_Y_all.shape,
        chronic_mask.shape,
        chronic_mask_targets.shape,
    )

    X_all.append(chronic_X_all)
    Y_all.append(chronic_Y_all)
    mask_all.append(chronic_mask)
    mask_targets.append(chronic_mask_targets)

X_all = np.stack(X_all).astype(np.float)
Y_all = np.stack(Y_all).astype(np.float)
mask_all = np.stack(mask_all).astype(np.bool)
mask_targets = np.stack(mask_targets).astype(np.bool)

X = X_all
Y = Y_all
mask = np.multiply(mask_all, mask_targets)

X_train, X_test, Y_train, Y_test, mask_train, mask_test = train_test_split(
    X, Y, mask, test_size=test_frac, random_state=random_seed
)
X_train, X_val, Y_train, Y_val, mask_train, mask_val = train_test_split(
    X_train, Y_train, mask_train, test_size=val_frac, random_state=random_seed
)

print_dataset(X_all, Y_all, mask_all, "All data")
print_dataset(X, Y, mask, "Data")
print_dataset(X_train, Y_train, mask_train, "Train")
print_dataset(X_val, Y_val, mask_val, "Validation")
print_dataset(X_test, Y_test, mask_test, "Test")

n_batch = 1

train = tf.data.Dataset.from_tensor_slices(((X_train, mask_train), Y_train)).batch(
    n_batch
)
val = tf.data.Dataset.from_tensor_slices(((X_val, mask_val), Y_val)).batch(n_batch)
test = tf.data.Dataset.from_tensor_slices(((X_test, mask_test), Y_test)).batch(n_batch)

100
100
0                                       (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
1                                       (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
3                                       (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
4                                       (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
7                                       (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
10                                      (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
11                                      (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
12                                      (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
13                                      (100, 246)	(7964, 246)	(8064, 246)	(8064,)	(8064,)	(8064,)
100
100
14                           

ValueError: could not broadcast input array from shape (88) into shape (89)

In [ ]:
"""
    Model
"""

metrics = [
    tf.keras.metrics.TruePositives(thresholds=threshold, name="tp"),
    tf.keras.metrics.FalsePositives(thresholds=threshold, name="fp"),
    tf.keras.metrics.TrueNegatives(thresholds=threshold, name="tn"),
    tf.keras.metrics.FalseNegatives(thresholds=threshold, name="fn"),
    tf.keras.metrics.BinaryAccuracy(threshold=threshold, name="accuracy"),
    tf.keras.metrics.Precision(thresholds=threshold, name="precision"),
    tf.keras.metrics.Recall(thresholds=threshold, name="recall"),
    MatthewsCorrelationCoefficient(threshold=threshold, name="mcc"),
]

n = mask.sum()
n_positive = Y.sum()
n_negative = n - n_positive

class_weight = {0: n / n_negative / 2.0, 1: n / n_positive / 2.0}
initial_bias = np.log([n_positive / n_negative])

print_class_weights(class_weight)
pprint("Initial bias:", "{:.4f}".format(float(initial_bias)))

tf.random.set_seed(random_seed)

n_input = X.shape[-1]


class RNNetworkBinary(tf.keras.Model):
    def __init__(
        self, input_network, output_network, n_state,
    ):
        super(RNNetworkBinary, self).__init__()

        self.n_state = n_state
        self.initial_state = None

        self.input_network = input_network
        self.output_network = output_network

        self.class_weight = None
        if class_weight:
            self.class_weight = dict()
            for c, weight in class_weight.items():
                self.class_weight[c] = tf.constant(weight, dtype=tf.float64)

        self.rnn = tf.keras.layers.LSTM(
            self.n_state, return_sequences=True, input_shape=(max_chronic_len, n_hidden)
        )

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, max_chronic_len, n_input], dtype=tf.float64),
            tf.TensorSpec(shape=[None, max_chronic_len], dtype=tf.bool),
            tf.TensorSpec(shape=(), dtype=tf.bool),
        ]
    )
    def call(self, x, mask, training=None):
        outputs = self.input_network(
            x, training=training
        )  # (None, seq_length, n_input)
        outputs = self.rnn(outputs, mask=mask, training=training, initial_state=None,)
        outputs = self.output_network(
            outputs, training=training
        )  # (None, seq_length, n_output)
        outputs = tf.reshape(outputs, [-1, max_chronic_len])
        return outputs

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float64)])
    def compute_sample_weight(self, y_true):
        if self.class_weight:
            sample_weight = tf.multiply(
                1.0 - y_true, self.class_weight[0]
            ) + tf.multiply(y_true, self.class_weight[1])
        else:
            sample_weight = tf.ones_like(y_true)

        sample_weight = tf.reshape(sample_weight, [1, -1])

        return sample_weight

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, max_chronic_len, n_input], dtype=tf.float64),
            tf.TensorSpec(shape=[None, max_chronic_len], dtype=tf.bool),
            tf.TensorSpec(shape=[None, max_chronic_len], dtype=tf.float64),
        ]
    )
    def train_step(self, x, mask, y):
        with tf.GradientTape() as gt:
            y_pred = self(x, mask, training=True)  # (None, seq_length, n_input)

            y = tf.reshape(y, [-1])  # (n_batch * seq_length, )
            y_pred = tf.reshape(y_pred, [-1])  # (n_batch * seq_length, )
            mask = tf.reshape(mask, [1, -1])  # (1, n_batch * seq_length)

            sample_weight = self.compute_sample_weight(y)
            sample_weight = tf.multiply(sample_weight, tf.cast(mask, tf.float64))

            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight=sample_weight,
                regularization_losses=self.losses,
            )

        grads = gt.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        self.compiled_metrics.update_state(y, y_pred, sample_weight=mask)
        return loss, {m.name: m.result() for m in self.metrics}

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, max_chronic_len, n_input], dtype=tf.float64),
            tf.TensorSpec(shape=[None, max_chronic_len], dtype=tf.bool),
            tf.TensorSpec(shape=[None, max_chronic_len], dtype=tf.float64),
        ]
    )
    def test_step(self, x, mask, y):
        y_pred = self(x, mask, training=False)

        y = tf.reshape(y, [-1])  # (n_batch * seq_length, )
        y_pred = tf.reshape(y_pred, [-1])  # (n_batch * seq_length, )
        mask = tf.reshape(mask, [1, -1])  # (1, n_batch * seq_length)

        sample_weight = self.compute_sample_weight(y)
        sample_weight = tf.multiply(sample_weight, tf.cast(mask, tf.float64))

        self.compiled_loss(
            y, y_pred, sample_weight=sample_weight, regularization_losses=self.losses
        )
        self.compiled_metrics.update_state(y, y_pred, sample_weight=mask)

        return {m.name: m.result() for m in self.metrics}

    def predict(self, data):
        y_pred = []

        for (x, mask), _ in data:
            preds = self(x, mask, training=tf.constant(True))
            y_pred.append(preds)

        y_pred = tf.concat(y_pred, axis=0)
        return y_pred

    def evaluate(self, data):
        self.reset_metrics()
        test_metrics = None
        for (x, mask), y in data:
            test_metrics = self.test_step(x, mask, y)

        return test_metrics

    def fit(self, data, epochs, validation_data=None):
        training = TrainingHistory()

        for epoch in range(epochs):
            start = time.time()

            self.reset_metrics()
            train_metrics = None
            for batch, ((x, mask), y) in enumerate(data):
                loss, train_metrics = self.train_step(x, mask, y)
                break

            training.update_history(train_metrics, epoch)

            if validation_data:
                val_metrics = model.evaluate(validation_data)
                training.update_history(val_metrics, epoch, "val_")

            pprint(
                "Epoch: {}/{}   {:.3f} s".format(epoch, epochs, time.time() - start),
                "loss: {:.8f} - val_loss: {:.4f}".format(
                    training["loss"][-1], training["val_loss"][-1]
                ),
            )

        return training
    
reg = None
input_network = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            n_hidden, input_shape=(max_chronic_len, n_input), kernel_regularizer=reg
        ),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, kernel_regularizer=reg),
        tf.keras.layers.Dropout(dropout_rate),
    ],
    name="input_network",
)

output_network = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            n_hidden, input_shape=(max_chronic_len, 2 * n_state), kernel_regularizer=reg
        ),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, kernel_regularizer=reg),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(
            1,
            activation="sigmoid",
            bias_initializer=tf.keras.initializers.Constant(initial_bias),
        ),
    ],
    name="output_network",
)

model = RNNetworkBinary(
    input_network=input_network, output_network=output_network, n_state=2 * n_state,
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=metrics,
)

In [ ]:
"""
    Training
"""

training = model.fit(train, epochs=2, validation_data=val)

In [ ]:
print_variables(model.trainable_variables)
plot_metrics(training, Y_train, Y_val, save_dir=None)